In [1]:
import sqlite3 as sqlite
import re

In [2]:
def createAllNews():
    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()
        
        cur.execute("drop table if exists allNews;")
        cur.execute("create table allNews(id integer primary key autoincrement, title text, link text unique, description text, published_at datetime, authors text, tags text, source text, orig_cluster integer)")
        
        con.commit()
        
#createAllNews()

In [3]:
# Create the regular expressions
reg1 = re.compile(r'\n') #Regex to replace <br /> with \n (see reg1.sub)
reg2 = re.compile(r'(<!--.*?-->|<[^>]*>)') #Regex to clean all html tags (anything with <something>)
reg3 = re.compile(r'&nbsp;') #Regex to clean all &nbsp 
reg4 = re.compile(r'&apos;') #Regex to clean all ' chars
reg5 = re.compile(r'https?:\/\/.*[\r\n]*') #Regex to remove all links

def cleanText(txt):
    first_filter = reg1.sub(' ', txt)
    second_filter = reg2.sub(' ', first_filter)
    third_filter = reg3.sub(' ', second_filter)
    fourth_filter = reg4.sub('', third_filter)
    return reg5.sub('', fourth_filter)

def integrateAllNews():
    allFeeds = []
    prev_date = "datetime('now','-1 days')"
    with sqlite.connect(r'news.db') as con: 
        cur = con.cursor()

        cur.execute("select title, link, description, published_at, authors, tags, source from rss where published_at > " + prev_date)
        rows = cur.fetchall()
        allFeeds += [(cleanText(x[0]), x[1], cleanText(x[2]), x[3], x[4], x[5], x[6]) for x in rows]

        cur.execute("select tweet, link, published_at, hashtags, source from twitter where published_at > " + prev_date)
        rows = cur.fetchall()
        allFeeds += [('', x[1], cleanText(x[0]), x[2], '', x[3], x[4]) for x in rows]

        cur.execute("select title, link, description, published_at, authors, tags, source from web where published_at > " + prev_date)
        rows = cur.fetchall()
        allFeeds += [(cleanText(x[0]), x[1], cleanText(x[2]), x[3], x[4], x[5], x[6]) for x in rows]

        cur.execute("select caption, link, published_at, source from instagram where published_at > " + prev_date)
        rows = cur.fetchall()
        allFeeds += [('', x[1], cleanText(x[0]), x[2], '', '', x[3]) for x in rows]
        
        cur.executemany("insert or ignore into allNews(title,link,description,published_at,authors,tags,source) values(?,?,?,?,?,?,?)", allFeeds)
        con.commit()

In [4]:
integrateAllNews()